In [3]:
from tqdm import tqdm
from glob import glob
from pathlib import Path
from Snapshot import fit_FFNN_checkpoint
from Helpers import Storage
from tensorflow import keras
import tensorflow as tf
import multiprocessing as mp
import pandas as pd
import pickle
import os
import time
import warnings
import logging
logging.getLogger('tensorflow').setLevel(logging.FATAL)


In [4]:

start = time.time()

# CPU Only
tf.config.set_visible_devices([], 'GPU')
# Number of CPU cores
num_cores = mp.cpu_count()

# Parameters
model = 'SnapKin'
pseudo = True
nam = ''
num_batches = 10
num_snapshots = 1
num_epochs = 100

if not pseudo:
    nam ='-NoPseudo'


run_classes = ['MTOR','MAPK1']
all_classes = ['MAPK1', 'MTOR']
# directory = 'Multi-{}'.format('-'.join(all_datasets))
directory = 'Folds'
data_dir = '../../Data-50/{}/'.format(directory)
save_dir = './Data-50{}/'.format(nam)
all_datasets = ['C2C12','ESC','MLC','NBC','L1-I','L1-F','L1-R','L6']



# FFNN Parameters
learning_rates = [0.01] # [ 0.0001, 0.001, 0.01, 0.1]
optimizer = keras.optimizers.SGD(momentum=0.9)
loss_fn = keras.losses.BinaryCrossentropy()
num_layers = 3
num_folds = 50
batch_sizes= [32] # [32,64]
verbose= 0
num_outputs = 1
tune = False
fold_sets = ['F{}'.format(i + 1) for i in range(num_folds)]


# Create save directory if it doesn't exist
Path(save_dir).mkdir(parents=True, exist_ok=True)
motifs = [x + '_motif' for x in all_classes]
fold_columns = ['F{}'.format(x) for x in range(1,num_folds+1)]
class_columns = ['y.{}'.format(x) for x in all_classes]
run_class_columns = ['y.{}'.format(x) for x in run_classes]

In [9]:
for dat in all_datasets:
    for clss, clss_column in zip(run_classes, run_class_columns):
        data_fp = '{}{}_{}_FLD.csv'.format(data_dir, dat, clss)
        data = [pd.read_csv(data_fp).drop_duplicates(subset=['site'])]
        check_dir = '{}{}_{}_{}/'.format(save_dir, model, dat, clss)
        Path(check_dir).mkdir(parents=True, exist_ok=True)

        folds = list(set(data[0]['F1'].to_list()))
        folds = [x for x in folds if x > -1] # Ignore negatives
        for fold_set in fold_sets:
            for fold in folds:
                ## Skip if file exists.
                save_fp = '{}{}_{}.pickle'.format(check_dir, fold_set, fold)
                checkpt_dir = '{}Saves/{}_{}/'.format(check_dir, fold_set, fold)
                print(checkpt_dir)

./Data-50/SnapKin_C2C12_MTOR/Saves/F1_0/
./Data-50/SnapKin_C2C12_MTOR/Saves/F1_1/
./Data-50/SnapKin_C2C12_MTOR/Saves/F1_2/
./Data-50/SnapKin_C2C12_MTOR/Saves/F1_3/
./Data-50/SnapKin_C2C12_MTOR/Saves/F1_4/
./Data-50/SnapKin_C2C12_MTOR/Saves/F2_0/
./Data-50/SnapKin_C2C12_MTOR/Saves/F2_1/
./Data-50/SnapKin_C2C12_MTOR/Saves/F2_2/
./Data-50/SnapKin_C2C12_MTOR/Saves/F2_3/
./Data-50/SnapKin_C2C12_MTOR/Saves/F2_4/
./Data-50/SnapKin_C2C12_MTOR/Saves/F3_0/
./Data-50/SnapKin_C2C12_MTOR/Saves/F3_1/
./Data-50/SnapKin_C2C12_MTOR/Saves/F3_2/
./Data-50/SnapKin_C2C12_MTOR/Saves/F3_3/
./Data-50/SnapKin_C2C12_MTOR/Saves/F3_4/
./Data-50/SnapKin_C2C12_MTOR/Saves/F4_0/
./Data-50/SnapKin_C2C12_MTOR/Saves/F4_1/
./Data-50/SnapKin_C2C12_MTOR/Saves/F4_2/
./Data-50/SnapKin_C2C12_MTOR/Saves/F4_3/
./Data-50/SnapKin_C2C12_MTOR/Saves/F4_4/
./Data-50/SnapKin_C2C12_MTOR/Saves/F5_0/
./Data-50/SnapKin_C2C12_MTOR/Saves/F5_1/
./Data-50/SnapKin_C2C12_MTOR/Saves/F5_2/
./Data-50/SnapKin_C2C12_MTOR/Saves/F5_3/
./Data-50/SnapKi

In [6]:
check_dir

'./Data-50/SnapKin_L6_MAPK1/'